<a href="https://colab.research.google.com/github/kethlyncampos/webscraper/blob/main/web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install requests
!pip install iso8601

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.7
    Uninstalling urllib3-1.26.7:
      Successfully uninstalled urllib3-1.26.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.0 requires urllib3[secure]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [1]:
import requests
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
import pytz
from datetime import datetime
import iso8601
from dateutil import tz
from google.colab import drive
from selenium.webdriver.common.by import By
import re


In [8]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-notifications")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36")
browser = webdriver.Chrome(options=options)

In [ ]:
def req_yahoo(url):
  browser.get(url)
  lastHeight = browser.execute_script("return document.body.scrollHeight")
  while True:
      browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      sleep(5)
      newHeight = browser.execute_script("return document.body.scrollHeight")
      if newHeight == lastHeight:
          break
      lastHeight = newHeight
  return BeautifulSoup(browser.page_source, 'html.parser')

In [3]:
def req_infomoney(url):
  try:
    browser.get(url)
    browser.implicitly_wait(10)
    browser.maximize_window()
    alert = browser.switch_to.active_element.click
    infinite = browser.find_element(By.ID, 'infinite-handle')
    browser.execute_script("arguments[0].click();", infinite)
    count = 40
    while(count > 0):
      browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      sleep(5)
      browser.execute_script("arguments[0].click();", infinite)
      newHeight = browser.execute_script("return document.body.scrollHeight")
      lastHeight = newHeight
      count-=1
  except:
    return False
  return BeautifulSoup(browser.page_source, 'html.parser')


In [4]:
def convert_timezone(date_time):
  sp = tz.gettz('America/Sao_Paulo')
  new_date = date_time.replace(tzinfo=pytz.utc)
  new_date = new_date.astimezone(sp)
  new_date = new_date.strftime('%Y-%m-%d %H:%M:%S')
  return new_date

In [5]:
def trata_string(data):
  data = re.sub(r'[\t\n ]+', ' ', data)
  data = re.sub(r'(?s)<figure.*?</figure>', ' ', data)
  return data

In [ ]:
page = req_yahoo('https://br.yahoo.com/topics/bolsa-de-valores/')

In [ ]:
links = page.find_all('a', attrs={'class':'js-content-viewer','class':'rapidnofollow'})

In [ ]:
lista_url = []
for link in links:
  aux = link['href']
  if 'http' not in aux:
    aux = 'https://br.yahoo.com' + aux
  if 'noticias-do-dia-' not in aux:
    lista_url.append(aux)
lista_url = set(lista_url)

In [10]:
title, description, author, publication_date, collect_date = "", "", "", "", ""

In [ ]:
noticias = []

for i in lista_url:
  temp = req_yahoo(i)
  title = temp.find('h1', attrs={'data-test-locator':'headline'}).text
  author = temp.find('span', attrs={'class':'caas-author-byline-collapse'}).text
  collect_date = convert_timezone(datetime.now())
  publication_date = convert_timezone(iso8601.parse_date(temp.find('time')['datetime']))
  description = temp.find('div', attrs={'class':'caas-body'}).text
  noticias.append(['Yahoo Finanças',title, description, author, publication_date, collect_date, i])

In [ ]:
dataset = pd.DataFrame(noticias,columns=['Fonte', 'Título', 'Descrição', 'Autor', 'Data/Hora de Publicação', 'Data/Hora de Coleta', 'URL'])

In [17]:
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
dataset.drop_duplicates(subset=['Descrição'], inplace = True)

In [ ]:
dataset.reset_index(drop = True, inplace = True)

In [ ]:
dataset.to_csv('/content/gdrive/MyDrive/news_yahoo.csv')

In [11]:
page = req_infomoney('https://www.infomoney.com.br/mercados/')
if not page:
  print("Recompile o código, aguarde um momento e tente novamente!")

Recompile o código, aguarde um momento e tente novamente!


In [10]:
links = page.find_all(class_="hl-title")

In [ ]:
lista_url = []
for j in links:
  aux = j.find('a')
  if(aux):
    if "stock-pickers" not in aux['href']:
      lista_url.append(aux['href'])
lista_url = set(lista_url)

In [ ]:
noticias = []
subtitle = ""
for k in lista_url:
  response = requests.get(k)
  temp = BeautifulSoup(response.content, 'html.parser')
  title = temp.find('h1', attrs={'class': 'page-title-1'})
  if title: # se não houver título, a página é do tipo cotações e possui apenas as informações sobre a empresa
    title = title.text
    subtitle = trata_string(temp.find('p', attrs={'class': 'article-lead'}).text)
    author = temp.find('span', attrs={'class':'author-name'})
    if author:
      author = trata_string(author.find('a').text) # se não houver autor, a página é do tipo guias
      collect_date = convert_timezone(datetime.now())
      publication_date = iso8601.parse_date(temp.find('time', attrs={'class':'entry-date'})['datetime']).strftime('%Y-%m-%d %H:%M:%S')
      description = trata_string(temp.find('div', attrs={'class':'article-content'}).text)
      noticias.append(['Infomoney',title, subtitle, description, author, publication_date, collect_date, k])

In [15]:
dataset2 = pd.DataFrame(noticias,columns=['Fonte', 'Título', 'Subtítulo','Descrição', 'Autor', 'Data/Hora de Publicação', 'Data/Hora de Coleta', 'URL'])

In [21]:
dataset2.drop_duplicates(subset=['Descrição'], inplace = True)

In [23]:
dataset2.reset_index(drop = True, inplace = True)

In [24]:
dataset2.to_csv('/content/gdrive/MyDrive/news_infomoney.csv')

In [ ]:
dataset2